# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import nltk
from sqlalchemy import create_engine
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report,f1_score

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
X = df['message']
Y = df.drop(['message','genre'],axis=1)

In [3]:
for i in Y.columns:
    print(Y[i].value_counts())

1    19874
Name: related, dtype: int64
0    15410
1     4464
Name: request, dtype: int64
0    19757
1      117
Name: offer, dtype: int64
1    10840
0     9034
Name: aid_related, dtype: int64
0    17794
1     2080
Name: medical_help, dtype: int64
0    18564
1     1310
Name: medical_products, dtype: int64
0    19150
1      724
Name: search_and_rescue, dtype: int64
0    19403
1      471
Name: security, dtype: int64
0    19015
1      859
Name: military, dtype: int64
0    19874
Name: child_alone, dtype: int64
0    18205
1     1669
Name: water, dtype: int64
0    16957
1     2917
Name: food, dtype: int64
0    17566
1     2308
Name: shelter, dtype: int64
0    19470
1      404
Name: clothing, dtype: int64
0    19271
1      603
Name: money, dtype: int64
0    19576
1      298
Name: missing_people, dtype: int64
0    19000
1      874
Name: refugees, dtype: int64
0    18682
1     1192
Name: death, dtype: int64
0    16433
1     3441
Name: other_aid, dtype: int64
0    18169
1     1705
Name: infrastruc

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    text=re.sub(r'[a-zA-Z0-9]',' ',text)
    words=word_tokenize(text.lower())
    stop=stopwords.words('english')
    words=[WordNetLemmatizer().lemmatize(word) for word in words if word not in stop]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('mult_clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,shuffle=True)
pipeline.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred=pipeline.predict(x_test)

In [8]:
for i,col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col],y_pred[:,i]))

related
             precision    recall  f1-score   support

          1       1.00      1.00      1.00      4969

avg / total       1.00      1.00      1.00      4969

request
             precision    recall  f1-score   support

          0       0.81      0.94      0.87      3837
          1       0.56      0.25      0.34      1132

avg / total       0.75      0.78      0.75      4969

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      4933
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      4969

aid_related
             precision    recall  f1-score   support

          0       0.53      0.42      0.47      2261
          1       0.59      0.69      0.63      2708

avg / total       0.56      0.56      0.56      4969

medical_help
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      4429
          1       0.23      0.02      0

In [9]:
(y_test==y_pred).mean()

related                   1.000000
request                   0.783659
offer                     0.992353
aid_related               0.564701
medical_help              0.886496
medical_products          0.929966
search_and_rescue         0.963775
security                  0.973637
military                  0.956933
child_alone               1.000000
water                     0.913866
food                      0.840411
shelter                   0.874421
clothing                  0.976454
money                     0.971423
missing_people            0.984705
refugees                  0.950896
death                     0.933387
other_aid                 0.816261
infrastructure_related    0.914872
transport                 0.933789
buildings                 0.924733
electricity               0.974240
tools                     0.991749
hospitals                 0.985510
shops                     0.991749
aid_centers               0.986919
other_infrastructure      0.942443
weather_related     

In [10]:
y_pred=pipeline.predict(x_train)
(y_train==y_pred).mean()

related                   1.000000
request                   0.826971
offer                     0.995505
aid_related               0.657095
medical_help              0.912378
medical_products          0.946796
search_and_rescue         0.967192
security                  0.978665
military                  0.960751
child_alone               1.000000
water                     0.928547
food                      0.879705
shelter                   0.902180
clothing                  0.983831
money                     0.975847
missing_people            0.987051
refugees                  0.963368
death                     0.949480
other_aid                 0.854076
infrastructure_related    0.924925
transport                 0.949279
buildings                 0.943710
electricity               0.978397
tools                     0.993291
hospitals                 0.987655
shops                     0.995572
aid_centers               0.985508
other_infrastructure      0.948474
weather_related     

In [11]:
for i,col in enumerate(y_train.columns):
    print(col)
    print(classification_report(y_train[col],y_pred[:,i]))

related
             precision    recall  f1-score   support

          1       1.00      1.00      1.00     14905

avg / total       1.00      1.00      1.00     14905

request
             precision    recall  f1-score   support

          0       0.84      0.96      0.90     11573
          1       0.71      0.38      0.50      3332

avg / total       0.81      0.83      0.81     14905

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     14824
          1       1.00      0.17      0.29        81

avg / total       1.00      1.00      0.99     14905

aid_related
             precision    recall  f1-score   support

          0       0.65      0.52      0.58      6773
          1       0.66      0.77      0.71      8132

avg / total       0.66      0.66      0.65     14905

medical_help
             precision    recall  f1-score   support

          0       0.91      1.00      0.95     13365
          1       0.88      0.18      0

In [12]:
pipeline.predict([x_train[6]])

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters ={'mult_clf__estimator__n_estimators':[10,50,100],
            'mult_clf__estimator__max_depth':[7,10,13,15],
            'mult_clf__estimator__criterion':['gini','entropy'],
            'mult_clf__estimator__min_samples_split':[2,3,4,5,6]}

cv = GridSearchCV(pipeline,param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'mult_clf__estimator__n_estimators': [10, 50, 100], 'mult_clf__estimator__max_depth': [7, 10, 13, 15], 'mult_clf__estimator__criterion': ['gini', 'entropy'], 'mult_clf__estimator__min_samples_split': [2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
cv.best_params_

{'mult_clf__estimator__criterion': 'gini',
 'mult_clf__estimator__max_depth': 15,
 'mult_clf__estimator__min_samples_split': 2,
 'mult_clf__estimator__n_estimators': 10}

In [16]:
y_pred=cv.predict(x_test)

In [17]:
for i,col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col],y_pred[:,i]))

related
             precision    recall  f1-score   support

          1       1.00      1.00      1.00      4969

avg / total       1.00      1.00      1.00      4969

request
             precision    recall  f1-score   support

          0       0.78      0.98      0.87      3837
          1       0.56      0.08      0.13      1132

avg / total       0.73      0.78      0.70      4969

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      4933
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      4969

aid_related
             precision    recall  f1-score   support

          0       0.56      0.37      0.45      2261
          1       0.59      0.76      0.67      2708

avg / total       0.58      0.58      0.57      4969

medical_help
             precision    recall  f1-score   support

          0       0.89      1.00      0.94      4429
          1       0.75      0.01      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [5]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('mult_clf',MultiOutputClassifier(AdaBoostClassifier()))
])

In [6]:
x_train,x_test,y_train,y_test=train_test_split(X,Y)
pipeline.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [7]:
y_pred=pipeline.predict(x_test)

In [8]:
for i,col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col],y_pred[:,i]))

related
             precision    recall  f1-score   support

          1       1.00      1.00      1.00      4969

avg / total       1.00      1.00      1.00      4969

request
             precision    recall  f1-score   support

          0       0.78      0.97      0.87      3822
          1       0.52      0.11      0.18      1147

avg / total       0.72      0.77      0.71      4969

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      4937
          1       0.00      0.00      0.00        32

avg / total       0.99      0.99      0.99      4969

aid_related
             precision    recall  f1-score   support

          0       0.58      0.29      0.38      2299
          1       0.57      0.82      0.67      2670

avg / total       0.57      0.57      0.54      4969

medical_help
             precision    recall  f1-score   support

          0       0.90      1.00      0.95      4472
          1       0.50      0.01      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
(y_test==y_pred).mean()

related                   1.000000
request                   0.771181
offer                     0.993560
aid_related               0.573154
medical_help              0.899980
medical_products          0.942846
search_and_rescue         0.963172
security                  0.978869
military                  0.955524
child_alone               1.000000
water                     0.915073
food                      0.851479
shelter                   0.884283
clothing                  0.979674
money                     0.972429
missing_people            0.985913
refugees                  0.956732
death                     0.945663
other_aid                 0.836788
infrastructure_related    0.912457
transport                 0.944657
buildings                 0.932985
electricity               0.972630
tools                     0.992353
hospitals                 0.985913
shops                     0.994365
aid_centers               0.982491
other_infrastructure      0.942644
weather_related     

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.